In [1]:
from difflib import SequenceMatcher
import regex as re
from tqdm import tqdm_notebook as tqdm
from collections import Counter

In [2]:
txt_ref_path = "DATA/SOMUN.TN.txt"
txt_trg_path = "DATA/DYBG.TN.txt"

with open(txt_ref_path, 'r', encoding="utf-8") as fl:
    txt_ref_raw = fl.read()

with open(txt_trg_path, 'r', encoding="utf-8") as fl:
    txt_trg_raw = fl.read()

In [3]:
# hanzi 범위
cjk_range = "[\p{Han}]"
cjk_range_re = re.compile( cjk_range, re.UNICODE)

# hanzi 추출
txt_ref = "".join( re.findall( cjk_range_re, txt_ref_raw  ) )
txt_trg = "".join( re.findall( cjk_range_re, txt_trg_raw  ) )

txt_ref_len = len( txt_ref )
txt_trg_len = len( txt_trg )

In [4]:
# params 

padding = 2
gram_n = 3
sim_cut_off = 0.7

In [5]:
def nGram( text, n=3 ):
    length = len( text )
    return [ text[i:i+n] for i in range( length - n +1 ) ]

In [20]:
def get_ranges( raw_idxs ,padding=1, gram_n=3, min_length=8 ):
    
    idxs = sorted( raw_idxs )
    range_begin = idxs[0]
    range_end = 0
    rst = []
    
    for k, i in enumerate( idxs ):
        if k == 1 : continue
        if idxs[k-1] in range( i-(padding+gram_n), i ):
            range_end = i
        else:
            if ( range_end - range_begin ) > min_length:
                rst.append( (range_begin, range_end ) )
            range_begin = i
    
    return rst

In [21]:
# text_ref
triGram = nGram( txt_ref, n=gram_n )
triGram_freq = Counter( triGram )
triGram_sort = triGram_freq.most_common()

In [22]:
ref_idxs = []
trg_idxs = []

for gram, freq in tqdm( triGram_sort ):
    if gram not in txt_trg: continue
    ref_idx = [ m.start() for m in re.finditer(gram, txt_ref) ]
    trg_idx = [ m.start() for m in re.finditer(gram, txt_trg) ]
    ref_idxs += ref_idx
    trg_idxs += trg_idx


In [28]:
ref_idxs = sorted( ref_idxs )
trg_idxs = sorted( trg_idxs )
print( len(ref_idxs ), len(trg_idxs ))

32261 47851


In [29]:
ref_ranges = get_ranges( ref_idxs, padding=padding ) 
trg_ranges = get_ranges( trg_idxs, padding=padding ) 
print( len(ref_ranges), len(trg_ranges ))

1076 1062


In [30]:
ref_ranges

[(36, 600),
 (646, 655),
 (685, 821),
 (827, 912),
 (920, 1216),
 (1368, 1449),
 (1462, 1475),
 (1488, 1497),
 (1554, 1568),
 (1596, 1608),
 (1671, 1706),
 (1725, 1765),
 (1777, 1885),
 (1928, 1961),
 (2024, 2124),
 (2138, 2154),
 (2172, 2189),
 (2203, 2213),
 (2233, 2414),
 (2464, 2473),
 (2503, 2595),
 (2632, 2645),
 (2655, 2695),
 (2701, 2724),
 (2732, 2764),
 (2784, 2856),
 (2872, 2943),
 (2951, 2967),
 (3022, 3041),
 (3079, 3100),
 (3137, 3157),
 (3196, 3213),
 (3308, 3319),
 (3330, 3343),
 (3388, 3405),
 (3442, 3482),
 (3509, 3574),
 (3592, 3628),
 (3634, 3693),
 (3701, 3715),
 (3721, 3738),
 (3745, 3805),
 (3873, 3888),
 (3895, 3912),
 (3922, 3970),
 (3985, 3998),
 (4004, 4063),
 (4071, 4083),
 (4090, 4142),
 (4157, 4172),
 (4182, 4235),
 (4242, 4322),
 (4335, 4400),
 (4406, 4472),
 (4507, 4555),
 (4582, 4596),
 (4647, 4661),
 (4668, 4682),
 (4688, 4709),
 (4715, 4745),
 (4763, 4782),
 (4863, 4896),
 (4918, 4935),
 (4949, 4962),
 (4968, 5055),
 (5085, 5115),
 (5150, 5252),
 (528

In [14]:
txt_ref[36:600]

'曰余聞上古之人春秋皆度百歲而動作不衰今時之人年半百而動作皆衰者時世異耶人將失之耶歧伯對曰上古之人其知道者法於陰陽和於術數食飮有節起居有常不妄作勞故能形與神俱而盡終其天年度百歲乃去今時之人不然也以酒爲漿以妄爲常醉以入房以欲竭其精以耗散其眞不知持滿不時御神務快其心逆於生樂起居無節故半百而衰也夫上古聖人之敎下也皆謂之虛邪賊風避之有時恬惔虛無眞氣從之精神內守病安從來是以志閑而少欲心安而不懼形勞而不倦氣從以順各從其欲皆得所願故美其食任其服樂其俗高下不相慕其民故曰朴是以嗜欲不能勞其目淫邪不能惑其心愚智賢不肖不懼於物故合於道所以能年皆度百歲而動作不衰者以其德全不危也帝曰人年老而無子者材力盡耶將天數然也歧伯曰女子七歲腎氣盛齒更髮長二七而天癸至任脈通太衝脈盛月事以時下故有子三七腎氣平均故眞牙生而長極四七筋骨堅髮長極身體盛壯五七陽明脈衰面始焦髮始墮六七三陽脈衰於上面皆焦髮始白七七任脈虛太衝脈衰少天癸竭地道不通故形壞而無子也丈夫八歲腎氣實髮長齒更二八腎氣盛天癸至精氣溢寫陰陽和故能有子三八腎氣平均筋骨勁强故眞牙生而長極四八筋骨隆盛肌肉滿壯五八腎氣衰髮墮齒槁六八陽氣衰竭於上面焦髮鬢頒白七八肝氣衰筋不能動天癸竭精少腎藏衰形體皆極八八則齒髮去腎者主水受五藏六府之精而藏之故五藏盛乃能寫今五藏皆衰筋骨解墮天癸盡矣故髮鬢白身體重行步不正而無子耳帝'

In [31]:
rst = []

for i_b, i_e in tqdm(ref_ranges):
    for j_b, j_e in trg_ranges:
        r = txt_ref[i_b:i_e+gram_n]
        t = txt_trg[j_b:j_e+gram_n]
        sim = SequenceMatcher(None,  r, t ).ratio()
        if sim <= sim_cut_off: continue
        rst.append( ( ( i_b, i_e+gram_n ), ( j_b, j_e+gram_n) )  )

In [32]:
style = """
<style>
.match { 
    color: blue;
}
</style>

"""

In [33]:
handler = open("rst.html", 'w', encoding="utf-8")
i = 1
border_size = 20
for ( i_b, i_e ), ( j_b, j_e)  in rst:    
    handler.write( "<div>" )
    handler.write( "\t<h2>{:d}</h2>\n".format(i) )
    handler.write( "\t<p class='ref'>{}<span class='match'>{}</span>{}</p>".format( txt_ref[i_b-border_size:i_b], txt_ref[i_b:i_e], txt_ref[i_e:i_e+border_size] ) )
    handler.write( "\t<p class='rtg'>{}<span class='match'>{}</span>{}</p>".format( txt_trg[j_b-border_size:j_b], txt_trg[j_b:j_e], txt_trg[j_e:j_e+border_size] ) )
    handler.write( "\n</div>" )
    handler.write( "<br><br>")
    i += 1

handler.write( style )
handler.close()

In [34]:
"abd" not in "berwabdafbe"

False

In [19]:
list( range(2,3) )

[2]